# Dependencies

In [1]:
import pandas as pd
import sqlite3

# Data Source
https://data.cnra.ca.gov/dataset/periodic-groundwater-level-measurements

# CSV ETL

### Import CSV's

In [4]:
#measurements_df = pd.read_csv('measurements.csv')
#perforations_df = pd.read_csv('perforations.csv')
stations_df = pd.read_csv('stations.csv')

C:\Users\jerry\AppData\Local\Temp\ipykernel_6816\1473397188.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  stations_df = pd.read_csv('stations.csv')


### Create year, month and day columns

In [5]:
measurements_df['year'] = measurements_df['msmt_date'].str.slice(0, 4).astype(int)
measurements_df['month'] = measurements_df['msmt_date'].str.slice(5, 7).astype(int)
measurements_df['day'] = measurements_df['msmt_date'].str.slice(8, 10).astype(int)

### Export CSV datasets to SQLite

In [6]:
conn = sqlite3.connect('Groundwater_DB.db')

# Measurements
measurements_df.to_sql('Measurements', conn, if_exists='replace', index=False)

# Stations
stations_df.to_sql('Stations', conn, if_exists='replace', index=False)

# Perforations
perforations_df.to_sql('Perforations', conn, if_exists='replace', index=False)

conn.close()

### Create view that combines Measurments, Stations and Perforations

In [7]:
conn = sqlite3.connect('Groundwater_DB.db')
cursor = conn.cursor()
create_view_sql = """
CREATE VIEW vw_groundwater AS
SELECT 
    M.year,
    M.site_code,
    M.gse_gwe,
    M.gwe,
    M.wlm_gse,
    M.wlm_rpe,
    S.latitude,
    S.longitude,
    S.gse,
    S.rpe,
    S.well_depth,
    P.top_prf_int AS p_top_prf_int,  -- Alias to avoid column name conflict
    P.bot_prf_int
FROM 
    Measurements M
LEFT JOIN 
    Stations S ON M.site_code = S.site_code
LEFT JOIN 
    Perforations P ON M.site_code = P.site_code;
"""
cursor.execute(create_view_sql)
conn.commit()
conn.close()

### Query SQLite Database vw_groundwater view

In [9]:
conn = sqlite3.connect('Groundwater_DB.db')
query = "SELECT * FROM vw_groundwater LIMIT 10"
df = pd.read_sql_query(query, conn)
conn.close()
df.head(10)

,year,site_code,gse_gwe,gwe,wlm_gse,wlm_rpe,latitude,longitude,gse,rpe,well_depth,p_top_prf_int,bot_prf_int
0,2023,320000N1140000W001,423.1,122.82,545.92,545.92,36.56048,-121.75463,545.92,545.92,386.0,230.0,374.0
1,2023,320000N1140000W001,383.5,162.42,545.92,545.92,36.56048,-121.75463,545.92,545.92,386.0,230.0,374.0
2,2023,320000N1140000W001,385.0,160.92,545.92,545.92,36.56048,-121.75463,545.92,545.92,386.0,230.0,374.0
3,2023,320000N1140000W001,377.0,168.92,545.92,545.92,36.56048,-121.75463,545.92,545.92,386.0,230.0,374.0
4,2023,320000N1140000W001,384.2,161.72,545.92,545.92,36.56048,-121.75463,545.92,545.92,386.0,230.0,374.0
5,2021,320000N1140000W001,412.0,133.92,545.92,545.92,36.56048,-121.75463,545.92,545.92,386.0,230.0,374.0
6,2021,320000N1140000W001,410.0,135.92,545.92,545.92,36.56048,-121.75463,545.92,545.92,386.0,230.0,374.0
7,2021,320000N1140000W001,411.0,134.92,545.92,545.92,36.56048,-121.75463,545.92,545.92,386.0,230.0,374.0
8,2021,320000N1140000W001,411.0,134.92,545.92,545.92,36.56048,-121.75463,545.92,545.92,386.0,230.0,374.0
9,2021,320000N1140000W001,409.0,136.92,545.92,545.92,36.56048,-121.75463,545.92,545.92,386.0,230.0,374.0
